### 데이터 로드 및 split

In [ ]:
import collections
import re

import pandas as pd
import numpy as np
import argparse


# set up arguments
parser = argparse.ArgumentParser(description='Split DataSet Arguments.')
parser.add_argument('--seed', dest='seed', type=int, help='seed argument')
parser.add_argument('--train', dest='train_propostion', type=float, help='seed argument', default="0.7")
parser.add_argument('--val', dest='val_propostion', type=float, help='seed argument', default="0.2")
parser.add_argument('--test', dest='test_propostion', type=float, help='seed argument', default="0.1")

args = parser.parse_args()

# Yelp 데이터 로드
csv_file = "../data/raw_train.csv"
review_subset = pd.read_csv(csv_file, delimiter=',')

by_rating = collections.defaultdict(list)

for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())

final_list = []
# 고정된 숫자로 설정
np.random.seed(args.seed)

# 분할 데이터 만들기 
for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)

    n_total = len(item_list)
    n_train = int(args.train_propostion * n_total) # 0.7
    n_val = int(args.val_propostion * n_total) # 0.2
    n_test = int(args.test_propostion * n_total) # 0.1

    # 데이터 나누기
    for item in item_list[:n_train]:
        item['split'] = 'train'

    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'

    for item in item_list[n_train+n_val:n_train+n_val+n_test]:
        item['split'] = 'test'

    final_list.extend(item_list)
    
# 최종 데이터 
final_reviews = pd.DataFrame(final_list)

# 데이터 정제
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]", r" ", text)
    return text

# 데이터 정제 적용하기
final_reviews.text = final_reviews.text.apply(preprocess_text)

# 데이터 csv로 저장하기
final_reviews.to_csv (r'../data/raw_split.csv', index = True, header=True)
